In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image size and batch size
img_size = (128, 128)
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_gen = train_datagen.flow_from_directory(
    "Dataset_Split/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Validation set
val_gen = val_datagen.flow_from_directory(
    "Dataset_Split/val",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Test set
test_gen = test_datagen.flow_from_directory(
    "Dataset_Split/test",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  # Important for consistent evaluation
)

Found 3500 images belonging to 2 classes.
Found 750 images belonging to 2 classes.
Found 750 images belonging to 2 classes.


In [7]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

base_model = ResNet50(
    input_shape = (128,128,3),
    include_top = False,
    weights='imagenet'
)

base_model.trainable = False

model = Sequential(
    [
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

In [9]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=callbacks
)

Epoch 1/50


/home/maharshi/.local/lib/python3.10/site-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
I0000 00:00:1754037420.131239    4163 service.cc:152] XLA service 0x7f2e2804e240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754037420.131294    4163 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2025-08-01 14:07:00.372383: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1754037422.185994    4163 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-01 14:07:03.275286: I external/local_xla/xla/service/gpu/autotuning

  1/110 ━━━━━━━━━━━━━━━━━━━━ 23:36 13s/step - accuracy: 0.3750 - loss: 0.0000e+00

I0000 00:00:1754037426.049212    4163 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 23/110 ━━━━━━━━━━━━━━━━━━━━ 15s 182ms/step - accuracy: 0.3769 - loss: 0.0000e+00

2025-08-01 14:07:12.515118: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.162 = (f32[12,64,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[12,64,32,32]{3,2,1,0} %bitcast.5132, f32[64,64,3,3]{3,2,1,0} %bitcast.5139, f32[64]{0} %bitcast.5141), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1_1/resnet50_1/conv2_block1_2_conv_1/convolution" source_file="/home/maharshi/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-08-01 14:07:12.752089: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.3894 - loss: 0.0000e+00

/home/maharshi/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-08-01 14:07:39.491071: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.162 = (f32[14,64,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[14,64,32,32]{3,2,1,0} %bitcast.4657, f32[64,64,3,3]{3,2,1,0} %bitcast.4664, f32[64]{0} %bitcast.4666), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1_1/resnet50_1/conv2_block1_2_conv_1/convolution" source_file="/home/maharshi/.local/lib/python3.10/

110/110 ━━━━━━━━━━━━━━━━━━━━ 50s 344ms/step - accuracy: 0.4000 - loss: 0.0000e+00 - val_accuracy: 0.4000 - val_loss: 0.0000e+00
Epoch 2/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.4046 - loss: 0.0000e+00

KeyboardInterrupt: 